In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import re

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/dataset.csv", encoding="latin-1")

In [ ]:
data.head()

,customer_query,intent,sentiment,responses,Unnamed: 4
0,I want to return my order,refund,negative,I'm sorry to hear that. Please provide your or...,NaN
1,Can I get a refund for this product?,refund,neutral,"Sure! To process a refund, please provide your...",NaN
2,The product I received is damaged,complaint,negative,I apologize for the inconvenience caused. Plea...,NaN
3,I'm very satisfied with my purchase,feedback,positive,Thank you for your feedback! We're glad to hea...,NaN
4,How can I track my order?,inquiry,neutral,"To track your order, please provide your order...",NaN


In [ ]:
one_hot = pd.get_dummies(data['intent'])
data.drop(['intent','sentiment','responses','Unnamed: 4'], axis=1, inplace=True)
data = pd.concat([data, one_hot], axis=1)
data.head()

,customer_query,complaint,feedback,inquiry,refund
0,I want to return my order,0,0,0,1
1,Can I get a refund for this product?,0,0,0,1
2,The product I received is damaged,1,0,0,0
3,I'm very satisfied with my purchase,0,1,0,0
4,How can I track my order?,0,0,1,0


In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    # removing contractions
    sentence = re.sub(r"i'm", "i am", sentence)
    sentence = re.sub(r"he's", "he is", sentence)
    sentence = re.sub(r"she's", "she is", sentence)
    sentence = re.sub(r"it's", "it is", sentence)
    sentence = re.sub(r"that's", "that is", sentence)
    sentence = re.sub(r"what's", "that is", sentence)
    sentence = re.sub(r"where's", "where is", sentence)
    sentence = re.sub(r"how's", "how is", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"can't", "cannot", sentence)
    sentence = re.sub(r"n't", " not", sentence)
    sentence = re.sub(r"n'", "ng", sentence)
    sentence = re.sub(r"'bout", "about", sentence)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    sentence = re.sub(r"[^a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence

In [ ]:
X = data["customer_query"].values
y = data.drop(["customer_query"], axis=1).values

max_features = 20000
maxlen = 150
vocab_size = 20000

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=150)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(4254, 150) (4254, 4)
(1824, 150) (1824, 4)


In [ ]:
import pickle

# Save the tokenizer to a file using pickle
with open('/content/drive/MyDrive/tokenizer1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(y)

[[0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 ...
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]]


In [ ]:
embed_dim = 100
num_heads = 4
ff_dim = 200

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(4, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test)
)

Epoch 1/10
34/34 [==============================] - 86s 2s/step - loss: 1.3928 - accuracy: 0.2755 - val_loss: 1.3742 - val_accuracy: 0.2703
Epoch 2/10
34/34 [==============================] - 75s 2s/step - loss: 1.3675 - accuracy: 0.2823 - val_loss: 1.3472 - val_accuracy: 0.4101
Epoch 3/10
34/34 [==============================] - 70s 2s/step - loss: 1.1149 - accuracy: 0.4974 - val_loss: 0.7114 - val_accuracy: 0.7248
Epoch 4/10
34/34 [==============================] - 73s 2s/step - loss: 0.4920 - accuracy: 0.8134 - val_loss: 0.2902 - val_accuracy: 0.9024
Epoch 5/10
34/34 [==============================] - 85s 3s/step - loss: 0.1704 - accuracy: 0.9499 - val_loss: 0.1752 - val_accuracy: 0.9507
Epoch 6/10
34/34 [==============================] - 74s 2s/step - loss: 0.1012 - accuracy: 0.9734 - val_loss: 0.1739 - val_accuracy: 0.9550
Epoch 7/10
34/34 [==============================] - 74s 2s/step - loss: 0.0844 - accuracy: 0.9793 - val_loss: 0.1097 - val_accuracy: 0.9759
Epoch 8/10
34/34 [==

In [ ]:
model.save("/content/drive/MyDrive/IntentClassificationTransformer.h5")

In [ ]:
text = ["Your staff's delivery time was accurate."]

text = preprocess_sentence(text[0])

text = tokenizer.texts_to_sequences(text)
text = pad_sequences(text, maxlen=150, dtype='int32', value=0)
intent = model.predict(text,batch_size=1,verbose = 2)[0]

print(intent)

if (np.argmax(intent) == 0):
  print("complaint")
elif (np.argmax(intent) == 1):
  print("feedback")
elif (np.argmax(intent) == 2):
  print("inquiry")
else:
  print("refund")

1/1 - 0s - 54ms/epoch - 54ms/step
[1.5064926e-03 9.9799013e-01 5.0150015e-04 1.8503110e-06]
feedback
